In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
import torch

In [4]:
import pyro
import deepppl

In [5]:
sns.set_style('whitegrid')


### Basic example showing the interface of *DeepPPL*
Similar to the coin model but this time using a `guide` to do stochastic variational inference


``` stan
data {
  int<lower=0,upper=1> x[10];
}
parameters {
  real<lower=0,upper=1> theta;
}


guide parameters
{
  real<lower=0>  alpha_q;
  real<lower=0>  beta_q;
}

guide {

  alpha_q = 15.0;
  beta_q = 15.0;
  theta ~ beta(alpha_q, beta_q);
}

model {
  theta ~ beta(10.0, 10.0);
  for (i in 1:10)
    x[i] ~ bernoulli(theta);
}
```

In [6]:
model = deepppl.DppplModel(model_file = '../tests/good/coin_guide.stan')

In [7]:
svi = model.svi()

Two positive and eight negative observations

In [8]:
x = torch.Tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0])

In [9]:
for step in range(10000):
    svi.step(x)
    if step % 100 == 0:
        print('.', end='')

....................................................................................................

In [10]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

We know that the posterior distribution should be a `Beta` distribution with parameters $$\alpha = 10 + 2$$ and $$\beta = 10 + 8$$

In [11]:
alpha_q, beta_q

(12.530339241027832, 17.1934814453125)

In [12]:
alpha_q/(alpha_q + beta_q)

0.42155883569793523

To do more experiments, the store must be cleared

In [13]:
pyro.clear_param_store()